In [1]:
import time
import pyscf
from pyscf import scf, gto, dft, cc
import numpy as np

In [ ]:
import numba as nb

In [29]:
import torch
import dqc

def DQC_HF(system, basis):
    atomzs, atomposs = dqc.parse_moldesc(system)
    atomposs
    mol = dqc.Mol((atomzs, atomposs), basis=basis)
    qc = dqc.HF(mol).run()
    ene = qc.energy()  # calculate the energy
    
    return ene

def DQC_KS_PBE(system, basis):
    atomzs, atomposs = dqc.parse_moldesc(system)
    atomposs
    mol = dqc.Mol((atomzs, atomposs), basis=basis)
    qc = dqc.KS(mol, xc="gga_c_pbe").run()
    ene = qc.energy() # calculate the energy
    return ene

def DQC_KS_LDA_X(system, basis):
    atomzs, atomposs = dqc.parse_moldesc(system)
    atomposs
    mol = dqc.Mol((atomzs, atomposs), basis=basis)
    qc = dqc.KS(mol, xc="lda_x").run()
    ene = qc.energy() # calculate the energy
    return ene

In [2]:
def PySCF_HF(system, basis):
    mol = gto.M(atom = system, basis = basis, unit="Bohr", verbose=0) # NB: PySCF uses Angstrom by default
    hf = scf.HF(mol)
    ene = hf.kernel(verbose=0)
    return ene

In [3]:
def PySCF_KS_PBE(system, basis):
    mol = gto.M(atom = system, basis = basis, unit="Bohr", verbose=0) # NB: PySCF uses Angstrom by default
    ks = dft.RKS(mol)
    ks.xc = "pbe"
    ene = ks.kernel()
    return ene

In [4]:
def PySCF_KS_LDA_X(system, basis):
    mol = gto.M(atom = system, basis = basis, unit="Bohr", verbose=0) # NB: PySCF uses Angstrom by default
    ks = dft.RKS(mol)
    ks.xc = "lda,"
    answer = ks.kernel()
    return answer

In [5]:
def PySCF_CCSDT(system, basis):
    mol = gto.M(atom = system, basis = basis, unit="Bohr", verbose=0) # NB: PySCF uses Angstrom by default
    hf = scf.HF(mol).run()
    cc_ = cc.CCSD(hf)
    ene = cc_.kernel()
    et = cc_.ccsd_t()
    return cc_.e_tot + et

In [6]:
def print_timer(t):
    print(f' took {round(time.time() - t, 3)} s.')

In [30]:
def accuracy_comparison_DQC(system, basis):
    t = time.time()
    print("HF, DQC: ", DQC_HF(system, basis))
    print_timer(t)
    t = time.time()
    print("PBE, DQC: ", DQC_KS_PBE(system, basis))
    print_timer(t)

In [7]:
def accuracy_comparison_PYSCF(system, basis):
    t = time.time()
    print("HF, PySCF: ", PySCF_HF(system, basis))
    print_timer(t)
    t = time.time()
    print("PBE, PySCF: ", PySCF_KS_PBE(system, basis))
    print_timer(t)
    t = time.time()
    print("CCSD(T): ", PySCF_CCSDT(system, basis))
    print_timer(t)

Hydrogen at equlibrium point:

In [35]:
accuracy_comparison_PYSCF("H 0 0 0; H 1.4 0 0", "6-31G")

HF, PySCF:  -1.1267427044517935
 took 0.069 s.
PBE, PySCF:  -1.1619157577671024
 took 0.074 s.
CCSD(T):  -1.1516791660767651
 took 0.067 s.


In [34]:
accuracy_comparison_DQC("H 0 0 0; H 1.4 0 0", "6-31G")

HF, DQC:  tensor(-1.1267, dtype=torch.float64)
 took 0.024 s.
PBE, DQC:  tensor(-0.5625, dtype=torch.float64)
 took 0.097 s.


In [41]:
%timeit DQC_HF("H 0 0 0; H 1.4 0 0", "6-31G")

13.4 ms ± 87.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
%timeit PySCF_HF("H 0 0 0; H 1.4 0 0", "6-31G")

57.7 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
%timeit DQC_KS_PBE("H 0 0 0; H 1.4 0 0", "6-31G")

444 ms ± 40.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%timeit PySCF_KS_PBE("H 0 0 0; H 1.4 0 0", "6-31G")

88.3 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Nitrogen at equlibrium point:

In [37]:
accuracy_comparison_PYSCF("N 0 0 0; N 2.07 0 0", "6-31G")

HF, PySCF:  -108.86787508005202
 took 0.076 s.
PBE, PySCF:  -109.3533492341709
 took 0.145 s.
CCSD(T):  -109.10264680238787
 took 0.176 s.


In [38]:
accuracy_comparison_DQC("N 0 0 0; N 2.07 0 0", "6-31G")

HF, DQC:  tensor(-108.8679, dtype=torch.float64)
 took 0.039 s.
PBE, DQC:  tensor(-96.5082, dtype=torch.float64)
 took 0.258 s.


In [ ]:
%timeit DQC_HF("N 0 0 0; N 2.07 0 0", "6-31G")

In [ ]:
%timeit PySCF_HF("N 0 0 0; N 2.07 0 0", "6-31G")

In [ ]:
%timeit DQC_KS_PBE("N 0 0 0; N 2.07 0 0", "6-31G")

In [ ]:
%timeit PySCF_KS_PBE("N 0 0 0; N 2.07 0 0", "6-31G")

Ammonia at equlibrium point:

In [10]:
accuracy_comparison_PYSCF("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

HF, PySCF:  -56.16102378396442
 took 0.08 s.
PBE, PySCF:  -56.451363472681926
 took 0.157 s.
CCSD(T):  -56.2921491649041
 took 0.274 s.


In [40]:
accuracy_comparison_DQC("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

HF, DQC:  tensor(-56.1610, dtype=torch.float64)
 took 0.039 s.
PBE, DQC:  tensor(-49.0140, dtype=torch.float64)
 took 0.459 s.


In [41]:
%timeit DQC_HF("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

16.5 ms ± 839 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit PySCF_HF("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

In [ ]:
%timeit DQC_KS_PBE("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

In [ ]:
%timeit PySCF_KS_PBE("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

Benzene at equlibrium point:

In [12]:
benzene_coords = """C        0.598362921      0.000000000     -4.742986733;
                    C       -0.557705772     -0.354690359     -4.044822733;
                    C        1.754431614      0.354690359     -4.044822733;
                    H       -1.457130878     -0.630640582     -4.587995733;
                    H        2.653856720      0.630640582     -4.587995733;
                    C       -0.557705772     -0.354690359     -2.648492733;
                    C        1.754431614      0.354690359     -2.648492733;
                    H       -1.457130878     -0.630640582     -2.105319733;
                    H        2.653856720      0.630640582     -2.105319733;
                    C        0.598362921      0.000000000     -1.950328733;
                    H        0.598362921      0.000000000     -0.863981733;
                    H        0.598362921      0.000000000     -5.829333733"""

In [13]:
accuracy_comparison_PYSCF(benzene_coords, "6-31G")

HF, PySCF:  -216.74175971291947
 took 0.206 s.
PBE, PySCF:  -218.0955795467098
 took 1.026 s.
CCSD(T):  -217.07797043623268
 took 3.377 s.


In [43]:
accuracy_comparison_DQC(benzene_coords, "6-31G")

HF, DQC:  tensor(-216.7411, dtype=torch.float64)
 took 2.238 s.
PBE, DQC:  tensor(-179.7154, dtype=torch.float64)
 took 9.126 s.


In [ ]:
%timeit DQC_HF(benzene_coords, "6-31G")

In [ ]:
%timeit PySCF_HF(benzene_coords, "6-31G")

In [ ]:
%timeit DQC_KS_PBE(benzene_coords, "6-31G")

In [ ]:
%timeit PySCF_KS_PBE(benzene_coords, "6-31G")

Naphtalene at equlibrium point:

In [15]:
naphtalene_coords = """H       -1.057571674      0.000000000     -4.361571149;
                    C       -1.057571674      0.487597890     -3.390034963;
                    C       -1.057571674      1.860916946     -3.299493774;
                    C       -1.057571674      2.493399263     -2.031832408;
                    C       -1.057571674      1.739878824     -0.880133713;
                    C       -1.057571674      0.320033769     -0.937967730;
                    C       -1.057571674     -0.320033769     -2.220831877;
                    C       -1.057571674     -1.739878824     -2.278665893;
                    C       -1.057571674     -2.493399263     -1.126967199;
                    C       -1.057571674     -1.860916946      0.140694168;
                    C       -1.057571674     -0.487597890      0.231235357;
                    H       -1.057571674      2.466184811     -4.201283252;
                    H       -1.057571674      3.577893185     -1.973126484;
                    H       -1.057571674      2.222891720      0.093690092;
                    H       -1.057571674      0.000000000      1.202771542;
                    H       -1.057571674     -2.466184811      1.042483645;
                    H       -1.057571674     -3.577893185     -1.185673122;
                    H       -1.057571674     -2.222891720     -3.252489699"""

In [16]:
accuracy_comparison_PYSCF(naphtalene_coords, "6-31G")

HF, PySCF:  -359.6605290903001
 took 1.085 s.
PBE, PySCF:  -361.9417375268738
 took 5.003 s.
CCSD(T):  -360.24270683546575
 took 39.031 s.


In [44]:
accuracy_comparison_DQC(naphtalene_coords, "6-31G")

HF, DQC:  tensor(-359.6521, dtype=torch.float64)
 took 18.503 s.
PBE, DQC:  tensor(-298.3350, dtype=torch.float64)
 took 51.88 s.


In [ ]:
%timeit DQC_HF(naphtalene_coords, "6-31G")

In [ ]:
%timeit PySCF_HF(naphtalene_coords, "6-31G")

In [ ]:
%timeit DQC_KS_PBE(naphtalene_coords, "6-31G")

In [ ]:
%timeit PySCF_KS_PBE(naphtalene_coords, "6-31G")

Anthracene at equlibrium point:

In [ ]:
anthracene_coords = """C        1.044178831     -0.528233237     -4.684523771;
C        1.044178831     -0.552713285     -3.254782652;
C        1.044178831     -1.749044624     -2.526815306;
C        1.044178831      0.704131514     -2.541488804;
C        1.044178831      0.692578077     -1.141127549;
C        1.044178831     -0.503753261     -0.413160203;
C        1.044178831     -1.760598060     -1.126454051;
C        1.044178831      0.658108897     -5.368920348;
C        1.044178831      1.919058175     -3.295618865;
H        1.044178831      1.638563069     -0.604255156;
C        1.044178831     -0.528233309      1.016580916;
C        1.044178831     -2.975524721     -0.372323990;
C        1.044178831     -2.954793300      0.997119891;
C        1.044178831     -1.714574573      1.700977987;
H        1.044178831      0.418178836      1.551045394;
H        1.044178831     -3.919661467     -0.910797789;
H        1.044178831     -3.885538136      1.556731594;
H        1.044178831     -1.717772077      2.786998206;
C        1.044178831      1.898326754     -4.665062745;
H        1.044178831     -1.474645383     -5.218988249;
H        1.044178831     -2.695029615     -3.063687699;
H        1.044178831      0.661305531     -6.454941061;
H        1.044178831      2.863194921     -2.757145066;
H        1.044178831      2.829071589     -5.224674449"""

In [ ]:
t = time.time()
print("HF, DQC: ", DQC_HF(anthracene_coords, "6-31G"))
print_timer(t)

In [ ]:
t = time.time()
print("HF, PySCF: ", PySCF_HF(anthracene_coords, "6-31G"))
print_timer(t)

In [ ]:
t = time.time()
print("PBE, PySCF: ", PySCF_KS_PBE(anthracene_coords, "6-31G"))
print_timer(t)

In [ ]:
t = time.time()
print("PBE, DQC: ", DQC_KS_PBE(anthracene_coords, "6-31G"))
print_timer(t)

In [ ]:
t = time.time()
print("CCSD(T): ", PySCF_CCSDT(anthracene_coords, "6-31G"))
print_timer(t)

Trying to set up HEG LDA functional:

In [19]:
from math import pi

factor = -3.0/4.0 * (3.0 / pi)**(1.0/3.0)
power = 4.0/3.0

In [20]:
water = """
O        0.000000000      0.000000000      0.000000000;
H        0.000000000      1.434938863      1.126357947;
H        0.000000000     -1.434938863      1.126357947
"""
ammonia = """
N 0.0 0.0 0.0; 
H 0.0 -1.772 -0.721; 
H 1.535 0.886 -0.721; 
H -1.535 0.886 -0.721
"""

In [45]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a, p):
        super().__init__()
        self.a = a
        self.p = p

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) 
                          + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a * rho ** self.p
a = torch.nn.Parameter(torch.tensor(factor, dtype=torch.double))
p = torch.nn.Parameter(torch.tensor(power, dtype=torch.double))
myxc = MyLDAX(a, p)

In [46]:
def dqc_compute(system, basis="6-31G", xc=myxc):
    mol = dqc.Mol(moldesc=system, basis=basis)
    qc = dqc.KS(mol, xc=xc).run()
    ene = qc.energy()
    return ene

**NB:**

`exc` is not $\epsilon * \rho$, but $\epsilon$.

`vrho` is not $\frac{d\epsilon}{d\rho}$, but $\frac{d\epsilon * \rho}{d\rho}$

In [21]:
def my_eval_xc(a,p):    
    def eval_xc(xc_code, rho, *args, **kwargs):
        exc = a * rho ** (p-1.0) # NB: it is epsilon, not epsilon * rho!
        vrho = a * p * rho ** (p-1.0)
        vgamma = None
        vlapl = None
        vtau = None
        vxc = (vrho, vgamma, vlapl, vtau)
        fxc = None  # 2nd order functional derivative
        kxc = None  # 3rd order functional derivative
        return exc, vxc, fxc, kxc
    return eval_xc

In [22]:
def scf_compute(system, basis="6-31G", xc = my_eval_xc(factor,power), verbose=0):
    scf_mol = pyscf.gto.M(atom = system, basis=basis, unit="Bohr")
    mf = pyscf.dft.RKS(scf_mol)
    mf = mf.define_xc_(xc, 'LDA')
    mf.verbose = verbose
    answer = mf.kernel()
    return answer

In [47]:
print("DQC, custom:", dqc_compute(ammonia))
print("DQC, from library:", DQC_KS_LDA_X(ammonia, "6-31G"))

DQC, custom: tensor(-55.4136, dtype=torch.float64, grad_fn=<AddBackward0>)
DQC, from library: tensor(-55.4136, dtype=torch.float64)


In [23]:
print("PySCF, custom:", scf_compute(ammonia, verbose=0))
print("PySCF, from library:", PySCF_KS_LDA_X(ammonia, "6-31G"))

PySCF, custom: -55.41360440678283
PySCF, from library: -55.41360440678283


In [49]:
print("DQC, custom:", dqc_compute(water))
print("DQC, from library:", DQC_KS_LDA_X(water, "6-31G"))

The 6-31G basis for atomz 8 does not exist, but we will download it
Downloaded to /home/rolandgrinis.linux/miniconda3/envs/noa/lib/python3.9/site-packages/dqc/api/.database/6-31g/08.gaussian94
DQC, custom: tensor(-75.1547, dtype=torch.float64, grad_fn=<AddBackward0>)
DQC, from library: tensor(-75.1547, dtype=torch.float64)


In [48]:
print("PySCF, custom:", scf_compute(water, verbose=0))
print("PySCF, from library:", PySCF_KS_LDA_X(water, "6-31G"))

PySCF, custom: -75.15470534035698
PySCF, from library: -75.1547053403571


In [50]:
print("DQC, custom:", dqc_compute(benzene_coords))
print("DQC, from library:", DQC_KS_LDA_X(benzene_coords, "6-31G"))

DQC, custom: tensor(-213.4143, dtype=torch.float64, grad_fn=<AddBackward0>)
DQC, from library: tensor(-213.4143, dtype=torch.float64)


In [25]:
print("PySCF, custom:", scf_compute(benzene_coords, verbose=0))
print("PySCF, from library:", PySCF_KS_LDA_X(benzene_coords, "6-31G"))

PySCF, custom: -213.41468421595175
PySCF, from library: -213.41468421598665


In [51]:
%timeit scf_compute(benzene_coords, verbose=0)

1.83 s ± 3.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
%timeit dqc_compute(benzene_coords)

5.39 s ± 38.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
